In [1]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense, Activation, Convolution2D, Flatten, MaxPooling2D, Dropout
from keras.utils import np_utils
from keras.layers.normalization import BatchNormalization
import os
import cv2

Using TensorFlow backend.


In [2]:
import keras
keras.__version__

'2.1.6'

In [3]:
import tensorflow as tf
tf.__version__

'1.11.0'

In [4]:
from tensorflow.python.client import device_lib
device_lib.list_local_devices()

[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 1801131043310183986, name: "/device:GPU:0"
 device_type: "GPU"
 memory_limit: 11281553818
 locality {
   bus_id: 1
   links {
   }
 }
 incarnation: 7632423017489348133
 physical_device_desc: "device: 0, name: Tesla K80, pci bus id: 0000:00:04.0, compute capability: 3.7"]

Import Dataset

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [7]:
!ls "/content/drive/My Drive/EmojiData"

angry  glass  kiss  smile


In [0]:
dataset_path = '/content/drive/My Drive/EmojiData/'

In [0]:
exp_data = []
labels = []
class_id = 0

In [0]:
data_x = []
data_y = []

In [11]:
ctr=0
for exp_name in os.listdir(dataset_path):
    label = ctr
    print(exp_name)
    if exp_name[0]!='.':
        photos = os.listdir(dataset_path+exp_name)
        for pic in photos:
#             print(pic)
            img = cv2.imread(dataset_path+exp_name+"/"+pic)
            data_x.append(img)
            data_y.append(label)
        ctr+=1

smile
angry
glass
kiss


In [12]:
data_x=np.array(data_x)
data_y=np.array(data_y)
data_x = data_x/255.0
print(data_x.shape)
print(data_y.shape)

(3960, 100, 100, 3)
(3960,)


In [13]:
print(data_y)
print(np.unique(data_y))

[0 0 0 ... 3 3 3]
[0 1 2 3]


In [14]:
data_y = np.array(data_y)
print(np.unique(data_y))

[0 1 2 3]


In [15]:
print(np.unique(data_y))
data_y = np_utils.to_categorical(data_y)

[0 1 2 3]


In [16]:
print(data_y)

[[1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 ...
 [0. 0. 0. 1.]
 [0. 0. 0. 1.]
 [0. 0. 0. 1.]]


In [17]:
model = Sequential()

model.add(Convolution2D(64, kernel_size=(3, 3), input_shape=(100, 100, 3)))
model.add(Activation('relu'))
model.add(BatchNormalization(axis=1))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add( Dropout(0.2) )
model.add(Convolution2D(128, kernel_size=(3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(BatchNormalization(axis=1))
model.add(Convolution2D(64, kernel_size=(4, 4)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(BatchNormalization(axis=1))
model.add( Dropout(0.2) )
model.add(Flatten())
model.add(Dense(256))
model.add(Activation('relu'))
model.add(Dense(4))
model.add(Activation('softmax'))
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 98, 98, 64)        1792      
_________________________________________________________________
activation_1 (Activation)    (None, 98, 98, 64)        0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 98, 98, 64)        392       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 49, 49, 64)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 49, 49, 64)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 47, 47, 128)       73856     
_________________________________________________________________
activation_2 (Activation)    (None, 47, 47, 128)       0         
__________

In [0]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [19]:
model.fit(data_x, data_y, epochs=5, batch_size=50)

Epoch 1/5
3960/3960 [==============================] - 10s 3ms/step - loss: 1.4049 - acc: 0.8369
Epoch 2/5
3960/3960 [==============================] - 8s 2ms/step - loss: 0.0219 - acc: 0.9942
Epoch 3/5
3960/3960 [==============================] - 8s 2ms/step - loss: 0.0176 - acc: 0.9952
Epoch 4/5
3960/3960 [==============================] - 8s 2ms/step - loss: 0.0130 - acc: 0.9962
Epoch 5/5
3960/3960 [==============================] - 8s 2ms/step - loss: 0.0081 - acc: 0.9980


In [0]:
model.save('/content/drive/My Drive/Face_Gesture_Recogizer_Final.h5')